<a href="https://colab.research.google.com/github/dohyung-kim/ccri/blob/main/script/adm0/pop_exposure_gee_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
import geemap
import pandas as pd
import os
import json
import geopandas as gpd
import geemap

# Initialize
ee.Authenticate()
ee.Initialize(project='unicef-ccri')

In [ ]:
# Define admin level
admin_level = 'adm0'

# Set output folder
output_folder = f'p1_exposure_{admin_level}'


In [ ]:

# Load child population and replace one image
org_childpop = ee.ImageCollection("projects/unicef-ccri/assets/childpop_constrained")
image_to_replace_id = 'tha_T_Under_18_2024_CN_100m_R2024A_v1'
new_image = ee.Image(f'projects/unicef-ccri/assets/{image_to_replace_id}').set('system:index', image_to_replace_id)
filtered_collection = org_childpop.filter(ee.Filter.neq('system:index', image_to_replace_id))
childpop = filtered_collection.merge(ee.ImageCollection([new_image])).mosaic()
scale = filtered_collection.first().projection().nominalScale()
totalpop = ee.Image("projects/unicef-ccri/assets/worldpop_1km")
totalpop_res = totalpop.projection().nominalScale()

reference_image = ee.Image("projects/unicef-ccri/assets/heatwave_frequency_2014_2023_avg")
target_scale = reference_image.projection().nominalScale()
target_crs = reference_image.projection().crs()

# Hazard list
hazards = [
    {"id": "projects/unicef-ccri/assets/river_flood_r100", "threshold": 0.01, "name": "river_flood_100yr_jrc_2024"},
    {"id": "projects/unicef-ccri/assets/coastal_flood_r100", "threshold": 0, "name": "coastal_flood_100yr_jrc_2024"},
    {"id": "projects/unicef-ccri/assets/storm_giri_rp100", "threshold": 17.5, "name": "tropical_storm_100yr_giri_2024"},
    {"id": "projects/unicef-ccri/assets/ASI_cropland_avg_2014_2023", "threshold": 30, "name": "agricultural_drought_fao_1984-2023"},
    {"id": "projects/unicef-ccri/assets/sma_copernicus_avg_2015_2024", "threshold": 0, "name": "drought_sma_copernicus_1984-2023"},
    {"id": "projects/unicef-ccri/assets/spi_copernicus_avg_2015_2024", "threshold": 0, "name": "drought_spi_copernicus_1984-2023"},
    {"id": "projects/unicef-ccri/assets/heatwave_frequency_2014_2023_avg", "threshold": 'Mean', "name": "heatwave_frequency_ecmwf_2014-2024"},
    {"id": "projects/unicef-ccri/assets/heatwave_duration_2014_2023_avg", "threshold": 'Mean', "name": "heatwave_duration_ecmwf_2014-2024"},
    {"id": "projects/unicef-ccri/assets/heatwave_severity_2014_2023_avg", "threshold": 'Mean', "name": "heatwave_severity_ecmwf_2014-2024"},
    {"id": "projects/unicef-ccri/assets/extreme_heat_days_2014_2023_avg", "threshold": 35, "name": "extreme_heat_ecmwf_2014-2024"},
    {"id": "projects/unicef-ccri/assets/FIRMS_MODIS_Mean_Annual_FRP_2001_2023", "threshold": 'Mean', "name": "fire_FRP_nasa_2001-2024"},
    {"id": "projects/unicef-ccri/assets/FIRMS_MODIS_Mean_Annual_Count_2001_2023", "threshold": 'Mean', "name": "fire_frequency_nasa_2001-2023"},
    {"id": "projects/unicef-ccri/assets/sand_dust_storm_annual", "threshold": 0, "name": "sand_dust_storm_unccd_2024"},
    {"id": "projects/unicef-ccri/assets/pm25_2013_2022_avg", "threshold": 5, "name": "air_pollution_pm25_2012-2022"},
    {"id": "projects/unicef-ccri/assets/Pv_average_2013_2022", "threshold": 0.001, "name": "vectorborne_malariapv_2012-2022"},
    {"id": "projects/unicef-ccri/assets/Pf_average_2013_2022", "threshold": 0.001, "name": "vectorborne_malariapf_2012-2022"}
]

# Load country boundaries
admin_fc_path = f'projects/unicef-ccri/assets/{admin_level}_wfp'
country_fc = ee.FeatureCollection(admin_fc_path).map(lambda f: f.setGeometry(f.geometry().simplify(100)))
simple_fc = ee.FeatureCollection(f'projects/unicef-ccri/assets/{admin_level}_simple')
adm_ids = country_fc.aggregate_array(f'{admin_level}_id').distinct().getInfo()
global_geom = simple_fc.geometry()

# Update thresholds if needed
for hazard in hazards:
    if hazard['threshold'] == 'Mean':
        layer = (
            ee.ImageCollection(hazard['id']).mosaic()
            if hazard['name'] in ["river_flood_100yr_jrc_2024", "coastal_flood_100yr_jrc_2024", "tropical_storm_100yr_giri_2024"]
            else ee.Image(hazard['id'])
        )
        th = layer.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=global_geom,
            scale=target_scale,
            bestEffort=True,
            maxPixels=1e13
        ).values().get(0)
        if th is not None:
            hazard['threshold'] = ee.Number(th).getInfo()
        else:
            print(f"⚠️ Skipping hazard {hazard['name']} — global mean threshold is null.")
            hazard['skip'] = True

# Loop through admin regions
for adm_id in adm_ids:
    print(adm_id)
    if adm_id is None:
        continue

    print(f"Submitting task for {admin_level}_id: {adm_id}...")
    region = country_fc.filter(ee.Filter.eq(f'{admin_level}_id', adm_id)).first()
    region_geom = region.geometry()
    iso3 = region.get('iso3')
    region_name = region.get(f'{admin_level}_name')

    childpop_sum = childpop.reduceRegion(
        ee.Reducer.sum(), region_geom, scale=scale, crs='EPSG:4326', maxPixels=1e13).get('b1')
    totalpop_sum = totalpop.reduceRegion(
        ee.Reducer.sum(), region_geom, scale=totalpop_res, crs='EPSG:4326', maxPixels=1e13).get('b1')

    features = []

    for hazard in hazards:
        if hazard.get('skip'):
            continue
        name = hazard['name']
        TH = hazard['threshold']

        layer = (
            ee.ImageCollection(hazard['id']).mosaic()
            if hazard['name'] in ["river_flood_100yr_jrc_2024", "coastal_flood_100yr_jrc_2024", "tropical_storm_100yr_giri_2024"]
            else ee.Image(hazard['id'])
        )

        if name == "agricultural_drought_fao_1984-2023":
            layer = layer.updateMask(layer.lte(100))
            exposed = childpop.updateMask(layer.gt(ee.Image.constant(TH)))
        elif name in ["drought_sma_copernicus_1984-2023", "drought_spi_copernicus_1984-2023"]:
            exposed = childpop.updateMask(layer.lt(ee.Image.constant(TH)))
        else:
            exposed = childpop.updateMask(layer.gt(ee.Image.constant(TH)))

        exposed_sum = exposed.reduceRegion(
            ee.Reducer.sum(), region_geom, scale=scale, crs='EPSG:4326', maxPixels=1e13).get('b1')

        feature = ee.Feature(None, {
            'iso3': iso3,
            f'{admin_level}_name': region_name,
            f'{admin_level}_id': adm_id,
            'hazard': name,
            'child_population_exposed': exposed_sum,
            'child_population_total': childpop_sum,
            'population_total': totalpop_sum
        })
        features.append(feature)

    task = ee.batch.Export.table.toDrive(
        collection=ee.FeatureCollection(features),
        description=f"{adm_id}_hazards_{admin_level}",
        folder=output_folder,
        fileFormat='CSV',
        selectors=[
            'iso3', f'{admin_level}_name', f'{admin_level}_id', 'hazard',
            'child_population_exposed', 'child_population_total', 'population_total'
        ]
    )
    task.start()

print("✅ All tasks submitted.")


5
Submitting task for adm0_id: 5...
251
Submitting task for adm0_id: 251...
9
Submitting task for adm0_id: 9...
30
Submitting task for adm0_id: 30...
87
Submitting task for adm0_id: 87...
95
Submitting task for adm0_id: 95...
39
Submitting task for adm0_id: 39...
48
Submitting task for adm0_id: 48...
55
Submitting task for adm0_id: 55...
101
Submitting task for adm0_id: 101...
168
Submitting task for adm0_id: 168...
178
Submitting task for adm0_id: 178...
197
Submitting task for adm0_id: 197...
207
Submitting task for adm0_id: 207...
244
Submitting task for adm0_id: 244...
258
Submitting task for adm0_id: 258...
268
Submitting task for adm0_id: 268...
1019
Submitting task for adm0_id: 1019...
1017
Submitting task for adm0_id: 1017...
1018
Submitting task for adm0_id: 1018...
98
Submitting task for adm0_id: 98...
149
Submitting task for adm0_id: 149...
281
Submitting task for adm0_id: 281...
288
Submitting task for adm0_id: 288...
3
Submitting task for adm0_id: 3...
7
Submitting task fo

In [ ]:
import ee
import geemap


# Create a map centered roughly on Afghanistan
Map = geemap.Map(center=[34, 66], zoom=6)

# Load Afghanistan geometry from your asset or GEE FeatureCollection
country = ee.FeatureCollection('projects/unicef-ccri/assets/admin0_wfp') \
            .filter(ee.Filter.eq('iso3', 'AFG')) \
            .first() \
            .geometry()

# Load the child population data
org_childpop = ee.ImageCollection("projects/unicef-ccri/assets/childpop_constrained")
imageToReplaceId = 'tha_T_Under_18_2024_CN_100m_R2024A_v1'
newImage = ee.Image(f'projects/unicef-ccri/assets/{imageToReplaceId}') \
              .set('system:index', imageToReplaceId)
filteredCollection = org_childpop.filter(ee.Filter.neq('system:index', imageToReplaceId))
childpop = filteredCollection.merge(ee.ImageCollection([newImage])).mosaic()

# Clip childpop to AFG
childpop_afg = childpop.clip(country)

# Add layers
Map.addLayer(country, {}, 'Afghanistan Boundary')
Map.addLayer(childpop_afg, {'min': 0, 'max': 100, 'palette': ['white', 'orange', 'red']}, 'ChildPop in AFG')

# Show map
Map


Map(center=[34, 66], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chil…

In [ ]:
scale = filteredCollection.first().projection().nominalScale().getInfo()
print(scale)

childpop_sum = childpop.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=country,  # not country.geometry()
    scale=scale,
    crs='EPSG:4326',
    maxPixels=1e13
)

print('Child population total in AFG:', childpop_sum.getInfo())


92.76624195666344
Child population total in AFG: {'b1': 21003580.26453929}
